In [1]:
from collections import defaultdict
from typing import Any, List, Set, Tuple

import dill
import numpy as np

from Settings import Settings
from crel_helper import get_cr_tags
from results_procesor import is_a_regular_code
from load_data import load_process_essays
from window_based_tagger_config import get_config
from wordtagginghelper import merge_dictionaries
from BrattEssay import ANAPHORA

DATASET = "SkinCancer" # CoralBleaching

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
# NOTE: These predictions are generated from the "./notebooks/SEARN/Keras - Train Tagger and Save CV Predictions For Word Tags.ipynb" notebook
# used as inputs to parsing model
coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

training_folder, test_folder

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


('/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Training/',
 '/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Test/')

In [2]:
train_fname = coref_output_folder + "training_crel_anatagged_essays.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)
    
print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

870 218


In [3]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
len(all_essays)

1088

In [4]:
num_words = 0
words_with_anaphora_tag = 0
words_with_cc = 0
cc_with_anaphora_tag = 0
words_with_crel = 0
words_with_crel_ana = 0

reg_code_sent_tally = defaultdict(int)
ana_code_sent_tally = defaultdict(int)

crel_tally = defaultdict(int)
ana_crel_tally = defaultdict(int)

for e in all_essays:
    for sent in e.sentences:
        unique_tags = set()
        
        for wd, tags in sent:
            
            num_words +=1
            has_cc_code = False
            has_crel_code = False
            has_crel_ana_code = False
            unique_tags.update(tags)
            
            for t in tags:

                t_lower = t.lower()
                if is_a_regular_code(t):
                    has_cc_code = True
                if "->" in t and ("other" not in t_lower and 
                                  "rhet" not in t_lower and 
                                  "change" not in t_lower):
                    has_crel_code = True
                    if ANAPHORA in t:
                        has_crel_ana_code = True

            if ANAPHORA in tags:
                words_with_anaphora_tag +=1
            
            if has_cc_code:
                words_with_cc += 1
                
            if has_crel_code:
                words_with_crel +=1
            
            if has_crel_ana_code:
                words_with_crel_ana +=1
                
            if ANAPHORA in tags and has_cc_code:
                cc_with_anaphora_tag +=1
        # end for
        for t in unique_tags:
            t_lower = t.lower()
            if "rhetorical" in t_lower or "other" in t_lower or "change" in t_lower:
                continue
            if is_a_regular_code(t):
                reg_code_sent_tally[t]+=1
            elif "->" not in t and "Anaphor:[" in t:
                ana_code_sent_tally[t] +=1
            if "->" in t:
                prefix = "Anaphor["
                if prefix in t:
                    k_fixed = t.replace(prefix, "").replace("]","")
                    if ANAPHORA not in k_fixed:
                        l,r = k_fixed.split("->")
                        l_code = l.replace("Causer:","")
                        r_code = r.replace("Result:","")
                        assert is_a_regular_code(l_code), l_code
                        assert is_a_regular_code(r_code), r_code
                        ana_crel_tally[t] +=1
                elif ANAPHORA not in t:
                    crel_tally[t] +=1
                    
num_words, words_with_anaphora_tag, cc_with_anaphora_tag, words_with_cc

(180873, 585, 0, 36689)

In [5]:
reg_code_sent_tally

defaultdict(int,
            {'1': 1342,
             '11': 176,
             '12': 169,
             '2': 1745,
             '3': 1146,
             '4': 718,
             '5': 2181,
             '50': 3024,
             '6': 631})

In [6]:
ana_code_sent_tally

defaultdict(int,
            {'Anaphor:[11]': 36,
             'Anaphor:[12]': 1,
             'Anaphor:[1]': 37,
             'Anaphor:[2]': 25,
             'Anaphor:[3]': 41,
             'Anaphor:[4]': 21,
             'Anaphor:[50]': 188,
             'Anaphor:[5]': 73,
             'Anaphor:[6]': 23})

In [7]:
crel_tally

defaultdict(int,
            {'Causer:1->Result:2': 638,
             'Causer:1->Result:3': 88,
             'Causer:1->Result:4': 5,
             'Causer:1->Result:5': 27,
             'Causer:1->Result:50': 589,
             'Causer:11->Result:12': 208,
             'Causer:11->Result:3': 14,
             'Causer:11->Result:4': 1,
             'Causer:11->Result:5': 5,
             'Causer:11->Result:50': 69,
             'Causer:12->Result:12': 1,
             'Causer:12->Result:2': 19,
             'Causer:12->Result:3': 293,
             'Causer:12->Result:4': 2,
             'Causer:12->Result:5': 5,
             'Causer:12->Result:50': 21,
             'Causer:2->Result:1': 3,
             'Causer:2->Result:11': 1,
             'Causer:2->Result:2': 2,
             'Causer:2->Result:3': 283,
             'Causer:2->Result:4': 83,
             'Causer:2->Result:5': 123,
             'Causer:2->Result:50': 709,
             'Causer:2->Result:6': 4,
             'Causer:3->Result:1

In [8]:
ana_crel_tally

defaultdict(int,
            {'Causer:1->Result:Anaphor[2]': 1,
             'Causer:1->Result:Anaphor[3]': 2,
             'Causer:1->Result:Anaphor[50]': 39,
             'Causer:11->Result:Anaphor[12]': 1,
             'Causer:11->Result:Anaphor[3]': 1,
             'Causer:11->Result:Anaphor[50]': 5,
             'Causer:12->Result:Anaphor[2]': 1,
             'Causer:12->Result:Anaphor[3]': 3,
             'Causer:12->Result:Anaphor[50]': 2,
             'Causer:2->Result:Anaphor[1]': 1,
             'Causer:2->Result:Anaphor[3]': 3,
             'Causer:2->Result:Anaphor[4]': 3,
             'Causer:2->Result:Anaphor[50]': 47,
             'Causer:2->Result:Anaphor[5]': 5,
             'Causer:3->Result:Anaphor[4]': 2,
             'Causer:3->Result:Anaphor[50]': 33,
             'Causer:3->Result:Anaphor[6]': 1,
             'Causer:4->Result:Anaphor[50]': 8,
             'Causer:4->Result:Anaphor[5]': 9,
             'Causer:5->Result:Anaphor[50]': 73,
             'Causer:5->R

In [9]:
sum_reg_code = sum(reg_code_sent_tally.values())
sum_ana_code = sum(ana_code_sent_tally.values())
sum_reg_code, sum_ana_code, round(100* sum_ana_code/(sum_ana_code+sum_reg_code),4)

(11132, 445, 3.8438)

# Words

In [10]:
print("Words with Anaphora tags:         {wds_ana_cnt},\t - {wds_ana:.4f}%".format(
    wds_ana_cnt=words_with_anaphora_tag, 
    wds_ana=100* words_with_anaphora_tag/num_words))
print("Concept codes with Anaphora tags: {cc_ana},\t - {cc_ana_pct:.4f} % of cc codes, \t{wds_ana_cc_pct:.4f} % of ana codes".format(
    cc_ana=cc_with_anaphora_tag,
    cc_ana_pct=100* cc_with_anaphora_tag/words_with_cc,
    wds_ana_cc_pct=100* cc_with_anaphora_tag/words_with_anaphora_tag))

Words with Anaphora tags:         585,	 - 0.3234%
Concept codes with Anaphora tags: 0,	 - 0.0000 % of cc codes, 	0.0000 % of ana codes


## Concept Codes (Sentences)

In [11]:
print("Anaphora Codes: {ana_code_cnt}, All Concept Codes: {cc_codes}\t - {pct_ana_codes:.2f}%".format(
    ana_code_cnt=sum_ana_code,
    cc_codes=(sum_reg_code+sum_ana_code),
    pct_ana_codes=100* sum_ana_code/(sum_ana_code+sum_reg_code)))

Anaphora Codes: 445, All Concept Codes: 11577	 - 3.84%


## Causal Relations (Sentences)

In [12]:
sum_reg_crel = sum(crel_tally.values())
sum_ana_crel = sum(ana_crel_tally.values())

In [13]:
print("Anaphora Crel Codes: {ana_crel_cnt}, All Crel Codes: {crel_codes}\t - {pct_ana_crel:.2f}%".format(
    ana_crel_cnt=sum_ana_crel,
    crel_codes=sum_ana_crel+sum_reg_crel,
    pct_ana_crel=100* sum_ana_crel/(sum_ana_crel+sum_reg_crel)))

Anaphora Crel Codes: 470, All Crel Codes: 7019	 - 6.70%
